In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import pickle


import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3])

os.chdir(r'H:\all\RL_Shrinkage_2024')
from helpers import helper_functions as hf
from ONE_YR.NonLinear_Shrinkage import regression_evaluation_funcs as re_hf
from helpers import eval_function_new



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\traitlets\con

#### Get the fixed Shrinkages Results

In [112]:
nonlinear_results_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\results"
AV_NL = pd.read_csv(f"{nonlinear_results_path}/AV_fixed_intensity.csv", index_col=0)
SD_NL = pd.read_csv(f"{nonlinear_results_path}/SD_fixed_intensity.csv", index_col=0)

linear_results_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results"
AV_Id = pd.read_csv(f"{linear_results_path}/AV_fixed_intensity.csv", index_col=0)
SD_Id = pd.read_csv(f"{linear_results_path}/SD_fixed_intensity.csv", index_col=0)


px.line(SD_NL.T)

In [113]:
px.line(SD_Id.T.iloc[2:, :])

In [114]:
SD_Id.idxmin(axis=1).to_dict()

{30: '0.13', 50: '0.2', 100: '0.23', 225: '0.24', 500: '0.24'}

# RL-Id (Identity Shrinkage) Results

#### Analyze Results from the Convex Combinations and create summary of results

In [155]:
oos_daily_rets = {}
convex_combo_rets = {}
for PF_SIZE in [30, 50, 100, 225, 500]:
    RES_PATH  = rf"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results\p{PF_SIZE}"
    res = pd.read_csv(RES_PATH + "/OOS_daily_returns.csv", index_col=0).values.reshape(-1,)
    oos_daily_rets[PF_SIZE] = res

    res = pd.read_csv(RES_PATH + "/Convex_Combo_DailyReturns.csv", index_col=0)
    convex_combo_rets[PF_SIZE] = res

# if not already saved, save them
oos_daily_rets = pd.DataFrame(oos_daily_rets)


In [156]:
oos_daily_rets.to_csv(r"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results" + r"\model_linear_daily_returns.csv")

##### OOS Results 

In [116]:
print("SD and AV OOS Results (with in-sample Cross-Validation):")

oos_linear_summary_res = pd.DataFrame({"AV": oos_daily_rets.mean() * (252) * 100,
              "SD": oos_daily_rets.std() * np.sqrt(252) * 100,
              "IR": (oos_daily_rets.mean() * (252) * 100) / (oos_daily_rets.std() * np.sqrt(252) * 100)
              })

oos_linear_summary_res

SD and AV OOS Results (with in-sample Cross-Validation):


,AV,SD,IR
30,10.542698,13.405457,0.786448
50,10.438513,13.282008,0.785914
100,8.580393,12.569699,0.682625
225,9.455752,11.449919,0.825836
500,9.210001,10.447056,0.881588


In [117]:
oos_linear_summary_res.to_csv(rf"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results\model_linear_summary_results.csv")

In [118]:
# create matrix containing SD's and AV's for convex combinations
AV_convex_combos = {}
SD_convex_combos = {}
for PF_SIZE in [30, 50, 100, 225, 500]:
    SD_convex_combos[PF_SIZE] = convex_combo_rets[PF_SIZE].std() * np.sqrt(252) * 100
    AV_convex_combos[PF_SIZE] = convex_combo_rets[PF_SIZE].mean() * (252) * 100

SD_convex_combos = pd.DataFrame(SD_convex_combos).round(2)
AV_convex_combos = pd.DataFrame(AV_convex_combos).round(2)
IR_convex_combos = pd.DataFrame(AV_convex_combos / SD_convex_combos).round(2)

In [152]:
RES_PATH

'H:\\all\\RL_Shrinkage_2024\\ONE_YR\\Linear_Shrinkage\\results\\p500'

In [153]:
SD_convex_combos.to_csv('H:\\all\\RL_Shrinkage_2024\\ONE_YR\\Linear_Shrinkage\\results' + "\\convex_combos_SD_summary.csv")
AV_convex_combos.to_csv('H:\\all\\RL_Shrinkage_2024\\ONE_YR\\Linear_Shrinkage\\results' + "\\convex_combos_AV_summary.csv")
IR_convex_combos.to_csv('H:\\all\\RL_Shrinkage_2024\\ONE_YR\\Linear_Shrinkage\\results' + "\\convex_combos_IR_summary.csv")

In [119]:
print("OOS Results of Convex Combination of RL-Id and optimal fixed shrinkage level. Also Result of OOS Results, \n"
      "re-scaled to have same mean as optimal fixed OOS level")
SD_convex_combos

OOS Results of Convex Combination of RL-Id and optimal fixed shrinkage level. Also Result of OOS Results, 
re-scaled to have same mean as optimal fixed OOS level


,30,50,100,225,500
0.0,13.41,13.28,12.57,11.45,10.45
0.1,13.38,13.25,12.57,11.43,10.43
0.2,13.36,13.25,12.57,11.42,10.41
0.3,13.35,13.24,12.57,11.40,10.40
0.4,13.34,13.24,12.57,11.39,10.40
0.5,13.34,13.23,12.57,11.38,10.39
0.6,13.33,13.23,12.57,11.38,10.39
0.7,13.33,13.23,12.57,11.37,10.40
0.8,13.33,13.24,12.58,11.37,10.40
0.9,13.33,13.24,12.59,11.37,10.42


##### Check Oracle Results and plot intensities over time

In [140]:
oos_model_intensities = {}
oos_oracle_intensities = {}
oos_oracle_daily_rets = {}

for PF_SIZE in [30, 50, 100, 225, 500]:
    RES_PATH  = rf"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results\p{PF_SIZE}"
    oracle = pd.read_csv(RES_PATH +  f"/oos_linear_oracle_intensity_p{PF_SIZE}.csv", index_col=0).values.reshape(-1,)
    oos_oracle_intensities[PF_SIZE] = oracle

    oracle_rets = pd.read_csv(RES_PATH +  f"/oos_linear_oracle_dailyrets_p{PF_SIZE}.csv", index_col=0).values.reshape(-1,)
    oos_oracle_daily_rets[PF_SIZE] = oracle_rets

    oos_model_intensities[PF_SIZE] = pd.read_csv(RES_PATH +  f"/oos_linear_model_intensity_p{PF_SIZE}.csv", index_col=0).values.reshape(-1,)


oos_model_linear_intensities = pd.DataFrame(oos_model_intensities)
oos_oracle_linear_daily_rets = pd.DataFrame(oos_oracle_daily_rets)
oos_oracle_linear_intensities = pd.DataFrame(oos_oracle_intensities)

In [157]:
oos_oracle_linear_daily_rets.to_csv(rf"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results\oracle_linear_daily_returns.csv")

In [121]:
#### for comparison; oracle SD and AV
print("Oracle SD and AV OOS Results (with in-sample Cross-Validation):")
oracle_identity_summary_results =  pd.DataFrame({"AV": oos_oracle_daily_rets.mean() * (252) * 100,
              "SD": oos_oracle_daily_rets.std() * np.sqrt(252) * 100,
              "IR": (oos_oracle_daily_rets.mean() * (252) * 100) / (oos_oracle_daily_rets.std() * np.sqrt(252) * 100)
              })
oracle_identity_summary_results

Oracle SD and AV OOS Results (with in-sample Cross-Validation):


,AV,SD,IR
30,12.305050,12.638590,0.973609
50,10.813104,12.614241,0.857214
100,10.215931,12.011623,0.850504
225,10.733642,10.880074,0.986541
500,10.301040,9.954382,1.034825


In [141]:
SUMMARY_RES_PATH  = rf"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results"
oracle_identity_summary_results.to_csv(SUMMARY_RES_PATH + "/oracle_linear_summary_results.csv")
oos_oracle_linear_intensities.to_csv(SUMMARY_RES_PATH + "/oracle_linear_intensities.csv")
oos_model_linear_intensities.to_csv(SUMMARY_RES_PATH + "/model_linear_intensities.csv")

In [123]:
### as comparison: results using cov1para intensities
PF_SIZE = 500


base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
# IMPORT SHRK DATASETS
pf_size = PF_SIZE  # DONT CHANGE HERE!!

fixed_shrk_name = 'cov1Para'
opt_shrk_name = 'cov1Para'
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
    fixed_shrk_data = pickle.load(f)
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\{opt_shrk_name}_factor-1.0_p{pf_size}.pickle", 'rb') as f:
    optimal_shrk_data = pickle.load(f)
c1p = optimal_shrk_data['shrk_factor'].values.astype(np.float64)

with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_rawres_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
    rawres_fixed_shrk_data = pickle.load(f)



In [124]:
c1p_rounded = np.round(c1p, 2)
c1p_rounded_oos = c1p_rounded[5040:]

x = np.diag(rawres_fixed_shrk_data.iloc[5040:].loc[:, c1p_rounded_oos])

x.std() * np.sqrt(252) * 100

np.float64(11.013846795392254)

In [147]:
oracle_linear_test = oos_oracle_linear_intensities[PF_SIZE].iloc[list(range(0,5313, 21))]
oracle_linear_test = np.repeat(oracle_linear_test, 21)
x = np.diag(rawres_fixed_shrk_data.iloc[5040:].loc[:, oracle_linear_test])
x.std() * np.sqrt(252) * 100

np.float64(9.953444960531833)

In [125]:
oos_model_intensities_short = oos_model_intensities.iloc[list(range(0,5313,21))]
px.line(oos_model_intensities_short, x=oos_model_intensities_short.index, y=oos_model_intensities_short.columns)

In [126]:
oos_oracle_intensities_short = oos_oracle_intensities.iloc[list(range(0,5313,21))]
px.line(oos_oracle_intensities_short, x=oos_oracle_intensities_short.index, y=oos_oracle_intensities_short.columns)

also save the base cov1para returns, as well as the cov1para shrinkages

In [63]:
returns = {}
cov1para_shrinkages = {}
for tmp_pf_size in [30, 50, 100, 225, 500]:
    with open(fr"H:\all\RL_Shrinkage_2024\ONE_YR\preprocessing\training_dfs\PF{tmp_pf_size}\cov1Para_factor-1.0_p{tmp_pf_size}.pickle", 'rb') as f:
        tmp = pickle.load(f)
    cov1para_shrinkages[tmp_pf_size] = tmp['shrk_factor']
    returns[tmp_pf_size] = tmp['pf_return']

dates = pd.to_datetime(tmp['date'], format="%Y%m%d")
returns  = pd.DataFrame(returns, dtype=np.float64)
returns.index = dates.values

cov1para_shrinkages = pd.DataFrame(cov1para_shrinkages, dtype=np.float64)
cov1para_shrinkages.index = dates.values
cov1para_shrinkages = cov1para_shrinkages.round(3)

In [98]:
from helpers import rl_covmat_ests_for_dataset as estimators
def eval_funcs_get_retuns(val_preds, rets_full, permnos, reb_days, val_indices):
    weighted_rets_model = []
    reb_date_1 = permnos.index[0]
    add_idx = np.where(rets_full.index == reb_date_1)[0][0]

    for i in range(len(val_indices)):
        idx = val_indices[i]
        past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21*12*1: idx + add_idx, :]
        past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
        past_ret_mat = past_ret_mat.fillna(0)
        fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]

        shrk, sample, target = estimators.cov2Para(past_ret_mat)
        model_covmat_est = val_preds[i] * target + (1-val_preds[i]) * sample

        weights_model = hf.calc_global_min_variance_pf(model_covmat_est)
        weighted_rets_model += list(fut_ret_mat @ weights_model)

    return weights_model, weighted_rets_model


In [90]:
base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
# IMPORT SHRK DATASETS
pf_size = 500  # DONT CHANGE HERE!!
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
rets_full = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")


fixed_shrk_name = 'cov1Para'
opt_shrk_name = 'cov1Para'
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
    fixed_shrk_data = pickle.load(f)
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\{opt_shrk_name}_factor-1.0_p{pf_size}.pickle", 'rb') as f:
    optimal_shrk_data = pickle.load(f)

with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_rawres_{fixed_shrk_name}_p{pf_size}.pickle", 'rb') as f:
    rawres_fixed_shrk_data = pickle.load(f)

# IMPORT FACTORS DATA AND PREPARE FOR FURTHER USE
factor_path = fr"{base_folder_path}\helpers"
factors = pd.read_csv(factor_path + "/all_factors.csv")
factors = factors.pivot(index="date", columns="name", values="ret")

# as our shrk data starts from 1980-01-15 our factors data should too
start_date = str(optimal_shrk_data['date'].iloc[0])
start_date = start_date[0:4] + '-' + start_date[4:6] + "-" + start_date[6:]
start_idx = np.where(factors.index == start_date)[0][0]
factors = factors.iloc[start_idx:start_idx+fixed_shrk_data.shape[0], :]

cov1para_shrk = optimal_shrk_data['shrk_factor'].values[5040:]

len_train = 5040
end_date = fixed_shrk_data.shape[0]
val_indices_correct = (len_train, end_date)
val_indices_results = [val_indices_correct[0] + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
val_idxes_shrkges = [0 + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]

In [94]:
cov1para_shrinkages[500].values[5040:]

array([0.073, 0.075, 0.077, ..., 0.058, 0.058, 0.058])

In [99]:
weights, daily_rets = eval_funcs_get_retuns(cov1para_shrinkages[500].values[5040:], rets_full, permnos, 0, val_indices_results)

In [65]:
# save them in results
tmp_res_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results"
cov1para_shrinkages.to_csv(tmp_res_path + "\\cov1para_intensities.csv")
returns.to_csv(tmp_res_path + "\\cov1para_daily_returns.csv")

# RL-NL results

In [160]:
oos_NL_daily_rets = {}
for PF_SIZE in [30, 50, 100, 225, 500]:
    NL_RES_PATH  = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\results"
    res = pd.read_csv(NL_RES_PATH +  f"/oos_NL_daily_rets_p{PF_SIZE}.csv", index_col=0).values.reshape(-1,)
    oos_NL_daily_rets[PF_SIZE] = res


# if not already saved, save them
oos_NL_daily_rets = pd.DataFrame(oos_NL_daily_rets)
oos_NL_daily_rets.to_csv(NL_RES_PATH + "/model_NL_daily_returns.csv")

In [150]:
oos_NL_daily_rets.std() * np.sqrt(252) * 100


30     13.419950
50     13.250486
100    12.621199
225    11.345314
500    10.358059
dtype: float64

In [128]:
print("SD and AV OOS Results (with in-sample Cross-Validation):")
oos_NL_summary_res = pd.DataFrame({"AV": oos_NL_daily_rets.mean() * (252) * 100,
              "SD": oos_NL_daily_rets.std() * np.sqrt(252) * 100,
              "IR": (oos_NL_daily_rets.mean() * (252) * 100) / (oos_NL_daily_rets.std() * np.sqrt(252) * 100)
              })
oos_NL_summary_res

SD and AV OOS Results (with in-sample Cross-Validation):


,AV,SD,IR
30,11.185101,13.419950,0.833468
50,10.534302,13.250486,0.795013
100,8.401440,12.621199,0.665661
225,9.780063,11.345314,0.862035
500,9.412993,10.358059,0.908760


In [129]:
oos_NL_summary_res.to_csv(NL_RES_PATH + "/model_NL_summary_results.csv")

##### Check Oracle Results and plot intensities over time

In [161]:
oos_model_intensities = {}
oos_oracle_intensities = {}
oos_oracle_daily_rets = {}

for PF_SIZE in [30, 50, 100, 225, 500]:
    NL_RES_PATH  = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\results"
    oracle = pd.read_csv(NL_RES_PATH +  f"/oos_NL_oracle_intensity_p{PF_SIZE}.csv", index_col=0).values.reshape(-1,)
    oos_oracle_intensities[PF_SIZE] = oracle

    oracle_rets = pd.read_csv(NL_RES_PATH +  f"/oos_NL_oracle_dailyrets_p{PF_SIZE}.csv", index_col=0).values.reshape(-1,)
    oos_oracle_daily_rets[PF_SIZE] = oracle_rets

    oos_model_intensities[PF_SIZE] = pd.read_csv(NL_RES_PATH +  f"/oos_NL_model_intensity_p{PF_SIZE}.csv", index_col=0).values.reshape(-1,)


oos_model_intensities = pd.DataFrame(oos_model_intensities)
oos_oracle_daily_rets = pd.DataFrame(oos_oracle_daily_rets)
oos_oracle_intensities = pd.DataFrame(oos_oracle_intensities)


In [162]:
oos_oracle_daily_rets.to_csv(NL_RES_PATH + "/oracle_NL_daily_returns.csv")

In [163]:
oos_oracle_daily_rets.std() * np.sqrt(252) * 100

30     13.276836
50     13.112437
100    12.455574
225    11.133293
500    10.163286
dtype: float64

In [165]:
#### for comparison; oracle SD and AV
print("Oracle SD and AV OOS Results (with in-sample Cross-Validation):")
oos_oracle_NL_summary_res = pd.DataFrame({"AV": oos_oracle_daily_rets.mean() * (252) * 100,
              "SD": oos_oracle_daily_rets.std() * np.sqrt(252) * 100,
              "IR": (oos_oracle_daily_rets.mean() * (252) * 100) / (oos_oracle_daily_rets.std() * np.sqrt(252) * 100)
              })


Oracle SD and AV OOS Results (with in-sample Cross-Validation):


In [167]:
oos_oracle_NL_summary_res

,AV,SD,IR
30,11.674107,13.276836,0.879284
50,10.835202,13.112437,0.826330
100,8.549847,12.455574,0.686427
225,10.284148,11.133293,0.923729
500,9.656627,10.163286,0.950148


In [166]:
## save everything
NL_RES_PATH  = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\results"
oos_oracle_NL_summary_res.to_csv(NL_RES_PATH + "/oracle_NL_summary_results.csv")
oos_oracle_intensities.to_csv(NL_RES_PATH + "/oracle_NL_intensities.csv")
oos_model_intensities.to_csv(NL_RES_PATH + "/model_NL_intensities.csv")

In [133]:
oos_oracle_NL_summary_res

,AV,SD,IR
30,10.542698,13.405457,0.786448
50,10.438513,13.282008,0.785914
100,8.580393,12.569699,0.682625
225,9.455752,11.449919,0.825836
500,9.210001,10.447056,0.881588


In [134]:
oos_oracle_intensities

,30,50,100,225,500
0,0.65,2.0,1.2,1.05,1.95
1,0.65,2.0,1.2,1.05,1.95
2,0.65,2.0,1.2,1.05,1.95
3,0.65,2.0,1.2,1.05,1.95
4,0.65,2.0,1.2,1.05,1.95
...,...,...,...,...,...
5308,1.25,2.0,2.0,2.00,1.75
5309,1.25,2.0,2.0,2.00,1.75
5310,1.25,2.0,2.0,2.00,1.75
5311,1.25,2.0,2.0,2.00,1.75


In [135]:
oos_model_intensities

,30,50,100,225,500
0,1.35,1.55,1.65,1.5,1.50
1,1.35,1.55,1.65,1.5,1.50
2,1.35,1.55,1.65,1.5,1.50
3,1.35,1.55,1.65,1.5,1.50
4,1.35,1.55,1.65,1.5,1.50
...,...,...,...,...,...
5308,1.35,1.45,1.45,1.5,1.25
5309,1.35,1.45,1.45,1.5,1.25
5310,1.35,1.45,1.45,1.5,1.25
5311,1.35,1.45,1.45,1.5,1.25


#### plot model intensities to double check

In [136]:
oos_model_intensities_short = oos_model_intensities.iloc[list(range(0,5313,21))]
px.line(oos_model_intensities_short, x=oos_model_intensities_short.index, y=oos_model_intensities_short.columns)